In [ ]:
import os
import matplotlib.pyplot as plt
from scipy.io import wavfile
import noisereduce
import numpy as np
import pandas as pd
import librosa
import skimage.io


directory = '../daps/cleanraw/'

def slice_audio(waveform, sample_rate, segment_length,filename):

    segment_length_frames = int(segment_length * sample_rate)
    num_segments = len(waveform) // segment_length_frames

    for i in range(num_segments):
        start_frame = i * segment_length_frames
        end_frame = start_frame + segment_length_frames
        slice_waveform = waveform[start_frame:end_frame]

        spectrogram_image(slice_waveform,sample_rate,f"../sliced_spectograms/{filename}_{i}.png",512,128)

def scale_minmax(X, min=0.0, max=1.0):
    X_std = (X - X.min()) / (X.max() - X.min())
    X_scaled = X_std * (max - min) + min
    return X_scaled

def spectrogram_image(y, sr, out, hop_length, n_mels):
    # use log-melspectrogram
    mels = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels,
                                            n_fft=hop_length*2, hop_length=hop_length)
    mels = np.log(mels + 1e-9) # add small number to avoid log(0)

    # min-max scale to fit inside 8-bit range
    img = scale_minmax(mels, 0, 255).astype(np.uint8)
    img = np.flip(img, axis=0) # put low frequencies at the bottom in image
    img = 255-img # invert. make black==more energy

    # save as PNG
    skimage.io.imsave(out, img)

#https://stackoverflow.com/questions/56719138/how-can-i-save-a-librosa-spectrogram-plot-as-a-specific-sized-image/57204349#57204349

for filename in os.listdir(directory):
    if filename.startswith('.'):
        continue
    if filename.endswith('.wav'):
        filepath = os.path.join(directory, filename)
        
        try:
            waveform, sample_rate = librosa.load(filepath)
            waveform_t, index = librosa.effects.trim(waveform,top_db=20) #trimming silence on both ends
            waveform_tr = noisereduce.reduce_noise(y=waveform_t,sr=sample_rate) # reducing noise
            
            base_filename = os.path.splitext(filename)[0]

            slice_audio(waveform_tr,sample_rate,3,base_filename)

            
        except ValueError as e:
            print(f"Error reading {filename}: {e}")
        except Exception as e:
            print(f"An unexpected error occurred with {filename}: {e}")